## Evaluation Assignment

Data: outbrain click prediction

Tasks:
Using Spark RDD, DataFrame API and Python, calculate:

**1**. Top 10 most visited document_ids in the page_views_sample log

**2**. How many users have at least 2 different traffic_sources in the page_views_sample log (note the value is not a count, it's an encoded enum)

**3***. Top 10 most visited topic_ids in page_views_sample log (use documents_topics table)

The submission format is the result.json json file with top_10_documents, users and top_10_topics keys.
For TOP-10 results, the answer must be written in the form of a sheet ordered from TOP-1 to TOP-10 with an id.

result.json example:

    {
        "top_10_documents": [
            111,
            222,
            333,
            ...,
            1010
        ],
        "users": 10000,
        "top_10_topics": [
            11,
            22,
            33,
            ...,
            101
        ]
    }

In [9]:
! /home/jovyan/start-hadoop.sh

jovyan
 * Starting OpenBSD Secure Shell server sshd
start-stop-daemon: unable to set gid to 0 (Operation not permitted)
   ...fail!
 * sshd is running
Starting namenodes on [localhost]
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
Starting datanodes
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
Starting secondary namenodes [8a8dc2b1f3c1]
8a8dc2b1f3c1: Warning: Permanently added '8a8dc2b1f3c1' (ED25519) to the list of known hosts.
Starting resourcemanager
Starting nodemanagers
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
30721 org.apache.hadoop.mapreduce.v2.hs.JobHistoryServer
27267 org.apache.spark.deploy.SparkSubmit pyspark-shell
27977 org.apache.spark.executor.YarnCoarseGrainedExecutorBackend --driver-url spark://CoarseGrainedScheduler@8a8dc2b1f3c1:43533 --executor-id 2 --hostname 8a8dc2b1f3c1 --cores 1 --app-id application_1680975987671_0005 --resourceProfil

In [2]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-04-08 20:05:27,202 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [11]:
!pip install kaggle --upgrade

  Using cached kaggle-1.5.13-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.3
    Uninstalling kaggle-1.5.3:
      Successfully uninstalled kaggle-1.5.3


In [3]:
! mkdir -p ~/.kaggle

In [13]:
%%writefile ~/.kaggle/kaggle.json
{"username":"mariafrolkina","key":"e9c8b62128ac29fc8bf06198f5b9d752"}

Overwriting /home/jovyan/.kaggle/kaggle.json


In [14]:
! chmod 600 ~/.kaggle/kaggle.json

In [15]:
! pip install -U urllib3 kaggle==1.5.3
! kaggle competitions download -c outbrain-click-prediction -f page_views_sample.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f documents_topics.csv.zip 

  Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
  Using cached kaggle-1.5.3-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.13
    Uninstalling kaggle-1.5.13:
      Successfully uninstalled kaggle-1.5.13
100%|███████████████████████████████████████▊| 148M/149M [00:05<00:00, 33.4MB/s]
100%|████████████████████████████████████████| 149M/149M [00:05<00:00, 28.5MB/s]
 98%|███████████████████████████████████████ | 118M/121M [00:10<00:01, 2.00MB/s]
100%|████████████████████████████████████████| 121M/121M [00:10<00:00, 12.6MB/s]


In [16]:
! unzip '*.zip'
! rm -rf *.zip

Archive:  documents_topics.csv.zip
replace documents_topics.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [19]:
! hdfs dfs -put page_views_sample.csv
! hdfs dfs -put documents_topics.csv

put: `page_views_sample.csv': File exists
put: `documents_topics.csv': File exists


In [4]:
! hadoop fs -ls -h

Found 3 items
drwxr-xr-x   - jovyan supergroup          0 2023-04-08 20:05 .sparkStaging
-rw-r--r--   1 jovyan supergroup    323.7 M 2023-04-08 18:37 documents_topics.csv
-rw-r--r--   1 jovyan supergroup    433.3 M 2023-04-08 18:36 page_views_sample.csv


In [8]:
page_views_sample = se.read.csv("page_views_sample.csv", header=True)
page_views_sample.show(5)

+--------------+-----------+---------+--------+------------+--------------+
|          uuid|document_id|timestamp|platform|geo_location|traffic_source|
+--------------+-----------+---------+--------+------------+--------------+
|1fd5f051fba643|        120| 31905835|       1|          RS|             2|
|8557aa9004be3b|        120| 32053104|       1|       VN>44|             2|
|c351b277a358f0|        120| 54013023|       1|       KR>12|             1|
|8205775c5387f9|        120| 44196592|       1|       IN>16|             2|
|9cb0ccd8458371|        120| 65817371|       1|   US>CA>807|             2|
+--------------+-----------+---------+--------+------------+--------------+
only showing top 5 rows



In [9]:
documents_topics = se.read.csv("documents_topics.csv", header=True)
documents_topics.show(5)

+-----------+--------+------------------+
|document_id|topic_id|  confidence_level|
+-----------+--------+------------------+
|    1595802|     140|0.0731131601068925|
|    1595802|      16|0.0594164867373976|
|    1595802|     143|0.0454207537554526|
|    1595802|     170|0.0388674285182961|
|    1524246|     113| 0.196450402209685|
+-----------+--------+------------------+
only showing top 5 rows



**1. Top 10 most visited document_ids in the page_views_sample log**

In [39]:
from pyspark.sql.functions import col

top_10_documents = page_views_sample.groupBy("document_id") \
                  .count() \
                  .sort(col("count").desc()) \
                  .limit(10)

In [40]:
top_10_documents.show()

+-----------+------+
|document_id| count|
+-----------+------+
|    1811567|429551|
|        234|179692|
|      42744|156231|
|    1858440|112140|
|    1780813|109624|
|      60164|104941|
|    1790442| 91420|
|    1877626| 80309|
|    1821895| 79956|
|     732651| 74630|
+-----------+------+



In [41]:
top_10_documents_list = [int(row['document_id']) for row in top_10_documents.collect()]

**2. How many users have at least 2 different traffic_sources in the page_views_sample log (note the value is not a count, it's an encoded enum)**

In [24]:
# number of records

page_views_sample.count()

9999999

In [25]:
# number of unique users

page_views_sample.select("uuid").distinct().count()

9202149

In [27]:
import pyspark.sql.functions as f

users_min2_traffic_sources = page_views_sample.groupBy("uuid") \
                            .agg(f.countDistinct("traffic_source") \
                            .alias("distinct_traffic_sources")) \
                            .filter("distinct_traffic_sources > 1") \
                            .count()

In [28]:
users_min2_traffic_sources

98080

**3. Top 10 most visited topic_ids in page_views_sample log (use documents_topics table)**

In [30]:
# joining two logs

joined = documents_topics.join(page_views_sample, "document_id", "inner")

In [42]:
top_10_topic_ids = joined.groupBy("topic_id") \
                  .count() \
                  .sort(col("count").desc()) \
                  .limit(10)

In [43]:
top_10_topic_ids.show()

+--------+-------+
|topic_id|  count|
+--------+-------+
|      20|1429253|
|      16|1425838|
|     216|1160563|
|     136|1099382|
|     140| 971983|
|     143| 919136|
|      36| 855793|
|      97| 839328|
|       8| 819599|
|     269| 727145|
+--------+-------+



In [44]:
top_10_topic_ids_list = [int(row['topic_id']) for row in top_10_topic_ids.collect()]

In [45]:
# saving the file with answers

import json

result = {
        "top_10_documents": top_10_documents_list,
        "users": users_min2_traffic_sources,
        "top_10_topics": top_10_topic_ids_list
}

with open("result.json", "w") as f:
    json.dump(result, f)

In [46]:
result

{'top_10_documents': [1811567,
  234,
  42744,
  1858440,
  1780813,
  60164,
  1790442,
  1877626,
  1821895,
  732651],
 'users': 98080,
 'top_10_topics': [20, 16, 216, 136, 140, 143, 36, 97, 8, 269]}

In [47]:
!curl -F file=@result.json "51.250.54.133:80/MDS-LSML1/airam01000110/w4/1"

1.0
Well done!
